# Reader

We've already explored the *reader* model in the first few sections of this chapter, where we loaded a pretrained model using the `transformers` library.

Now, we'll do the same thing but via Haystack - which uses the same `transformers` library in the background (so we can use the same pretrained model names).

To initialize our reader via Haystack all we need is:

In [2]:
from haystack.nodes import FARMReader
reader = FARMReader(model_name_or_path='deepset/bert-base-cased-squad2', use_gpu=True)

Now we have our reader model initialized, let's load up our FAISS index and DPR just like before.

In [5]:
from haystack.document_stores.faiss import FAISSDocumentStore
from haystack.nodes import DensePassageRetriever

path = '../../models/faiss'

# load FAISS from file (the squad validation set index)
document_store = FAISSDocumentStore.load(index_path=f'{path}/squad_dev.faiss')

# initialize DPR model
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
    passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
    use_gpu=True,
    embed_title=True
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


Haystack comes with a very convenient `ExtractiveQAPipeline` class which allows us to pass our `reader` and `retriever` to build an easy-to-use *extractive* Q&A pipeline:

In [8]:
from haystack.pipelines import ExtractiveQAPipeline

pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever)

We can ask questions using the `run` method, alongside the `query` parameter:

In [10]:
pipeline.run(query='What does theoretical computer science cover?')

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

{'query': 'What does theoretical computer science cover?',
 'no_ans_gap': 6.869088649749756,
 'answers': [<Answer {'answer': 'analysis of algorithms and computability theory', 'type': 'extractive', 'score': 0.8070334196090698, 'context': ' related fields in theoretical computer science are analysis of algorithms and computability theory. A key distinction between analysis of algorithms ', 'offsets_in_document': [{'start': 59, 'end': 106}], 'offsets_in_context': [{'start': 52, 'end': 99}], 'document_ids': ['42b3e25793765b085da634955c175f98'], 'meta': {'vector_id': '254'}}>,
  <Answer {'answer': 'algorithmic problems', 'type': 'extractive', 'score': 0.4293513298034668, 'context': 'fore the actual research explicitly devoted to the complexity of algorithmic problems started off, numerous foundations were laid out by various resea', 'offsets_in_document': [{'start': 67, 'end': 87}], 'offsets_in_context': [{'start': 65, 'end': 85}], 'document_ids': ['27a839b0a3dba79e809de956f6832be6'], 'met

Here we're returning a lot of different answers, which is not really necessary. We can limit the number of answers we return using two parameters, `top_k_retriever` and `top_k_reader`. Both of these limit the number of items being returned from the `retriever` and `reader` models respectively.

In [13]:
pipeline.run(query='What does theoretical computer science cover?',
             params={"Retriever": {"top_k": 5}})

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

{'query': 'What does theoretical computer science cover?',
 'no_ans_gap': 6.869073867797852,
 'answers': [<Answer {'answer': 'analysis of algorithms and computability theory', 'type': 'extractive', 'score': 0.8070324659347534, 'context': ' related fields in theoretical computer science are analysis of algorithms and computability theory. A key distinction between analysis of algorithms ', 'offsets_in_document': [{'start': 59, 'end': 106}], 'offsets_in_context': [{'start': 52, 'end': 99}], 'document_ids': ['42b3e25793765b085da634955c175f98'], 'meta': {'vector_id': '254'}}>,
  <Answer {'answer': 'algorithmic problems', 'type': 'extractive', 'score': 0.42935115098953247, 'context': 'fore the actual research explicitly devoted to the complexity of algorithmic problems started off, numerous foundations were laid out by various resea', 'offsets_in_document': [{'start': 67, 'end': 87}], 'offsets_in_context': [{'start': 65, 'end': 85}], 'document_ids': ['27a839b0a3dba79e809de956f6832be6'], 'me

And the `reader`:

In [14]:
pipeline.run(query='What does theoretical computer science cover?',
              params={"Reader": {"top_k": 3}})

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

{'query': 'What does theoretical computer science cover?',
 'no_ans_gap': 6.869088649749756,
 'answers': [<Answer {'answer': 'analysis of algorithms and computability theory', 'type': 'extractive', 'score': 0.8070334196090698, 'context': ' related fields in theoretical computer science are analysis of algorithms and computability theory. A key distinction between analysis of algorithms ', 'offsets_in_document': [{'start': 59, 'end': 106}], 'offsets_in_context': [{'start': 52, 'end': 99}], 'document_ids': ['42b3e25793765b085da634955c175f98'], 'meta': {'vector_id': '254'}}>,
  <Answer {'answer': 'algorithmic problems', 'type': 'extractive', 'score': 0.4293513298034668, 'context': 'fore the actual research explicitly devoted to the complexity of algorithmic problems started off, numerous foundations were laid out by various resea', 'offsets_in_document': [{'start': 67, 'end': 87}], 'offsets_in_context': [{'start': 65, 'end': 85}], 'document_ids': ['27a839b0a3dba79e809de956f6832be6'], 'met